In [ ]:
!pip install transformers==4.55.3 streamlit==1.24.0 sentencepiece==0.1.99 accelerate==0.24.1 transformers_stream_generator==0.0.4 tiktoken einops oss2 requests modelscope

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# from openxlab.model import download
from modelscope import snapshot_download
import json

# download model in openxlab
# model_name_or_path =download(model_repo='ajupyter/EmoLLM_internlm2_7b_full',
#         output='EmoLLM_internlm2_7b_full')

# download model in modelscope
# model_name_or_path = snapshot_download('chg0901/EmoLLM-InternLM7B-base')
model_name_or_path = snapshot_download('chg0901/EmoLLMV3.0') # use latest version
# offline model
# model_name_or_path = "/root/StableCascade/emollm2/EmoLLM/xtuner_config/merged"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map='auto')
model = model.eval()

system_prompt = '你是心理健康助手EmoLLM，由EmoLLM团队打造。你旨在通过专业心理咨询，协助来访者完成心理诊断。请充分利用专业心理学知识与咨询技术，一步步帮助来访者解决心理问题。'

def get_dialogue_history(dialogue_history_list: list):

    dialogue_history_tmp = []
    for item in dialogue_history_list:
        if item['role'] == 'counselor':
            text = '心理咨询师：'+ item['content']
        else:
            text = '用户：'+ item['content']
        dialogue_history_tmp.append(text)

    dialogue_history = '\n'.join(dialogue_history_tmp)

    return dialogue_history

with open('./auto_eval_chat_data.json') as f:
    data = json.load(f)

    for i in range(100):
        print(f'{i}'.center(50, '*'))
        dialogue_history_with_label = []
        dialogue_history_with_output = []

        messages_1 = [(system_prompt, '')]
        messages_2 = [(system_prompt, '')]

        dialogue = data[i]['messages']

        for j in range(0,10,2):
            usr_msg = dialogue[j]['content']
            dialogue_history_with_label.append({
                'role': 'client',
                'content': usr_msg
            })
            dialogue_history_with_output.append({
                'role': 'client',
                'content': usr_msg
            })

            response, history = model.chat(tokenizer, usr_msg, history=messages_1)
            dialogue_history_with_output.append({
                'role': 'counselor',
                'content': response
            })
            messages_1.append(('用户：' + usr_msg, '心理咨询师：'+ response))
            print(dialogue_history_with_output)
            dialogue[j+1]['EmoLLM_response_history_with_output'] = response

            response, history = model.chat(tokenizer, usr_msg, history=messages_2)
            dialogue_history_with_label.append({
                'role': 'counselor',
                'content': dialogue[j+1]['content']
            })
            messages_2.append(('用户：' + usr_msg, '心理咨询师：'+ response))
            dialogue[j+1]['EmoLLM_response_history_with_label'] = response

with open('./EmoLLM_result.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)